In [1]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import os
import re
import nltk
import nltk
from nltk.corpus import stopwords
import sklearn
from nltk.stem import PorterStemmer
from stemming.porter2 import stem

# load our pos/neg corpus and data

In [2]:
pos=list(set(open("positive.txt", 'r').read().split('\',\'')))

In [3]:
neg=list(set(open("negative.txt", 'r').read().split('\',\'')))

In [4]:
pos.remove('poor')

In [5]:
for i in range(len(pos)):
    temp=str('NOT_'+str(pos[i]))
    neg.append(temp)

In [6]:
data = [] 
cf = open('bars_rev_tr_df.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

In [7]:
id = open('uniq_id_lvbars.txt','r')
id_str=id.read()
id.close()

In [8]:
id_list=id_str.split(',')

In [9]:
dat_night=[]
for i in range(len(data)):
    if data[i]['business_id'] in id_list:
        dat_night.append(data[i])

In [10]:
dat_night_5star=[]
dat_night_4star=[]
dat_night_3star=[]
dat_night_2star=[]
dat_night_1star=[]
for i in range(len(dat_night)):
    if dat_night[i]['stars']=='5':
        dat_night_5star.append(dat_night[i])
    elif dat_night[i]['stars']=='4':
        dat_night_4star.append(dat_night[i])
    elif dat_night[i]['stars']=='3':
        dat_night_3star.append(dat_night[i])
    elif dat_night[i]['stars']=='2':
        dat_night_2star.append(dat_night[i])
    else:
        dat_night_1star.append(dat_night[i])



# Extract adj for star level 5

In [11]:
reviewslist = copy.deepcopy(dat_night_5star)
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)


for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    
    texts = re.sub('n\'t',' not', texts)
    
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    texts = re.sub('won\'t','will not', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    text_clean=preprocess_reviews(sent)
    rvs2 = " ".join([stem(word) for word in text_clean.split(" ")])
    
    reviewsTEXT_clean[ind]['text']=rvs2
    reviewsTEXT_clean[ind]['text_split']=rvs2.split()
    
#reviewsTEXT_clean[0]

## cocktail

In [12]:
count_pos=0
count_neg=0
list_pos=[]
list_neg=[]
    
for i in range(len(reviewsTEXT_clean)):
    if 'cocktail' in reviewsTEXT_clean[i]['text_split']:
#        print(reviewsTEXT_clean[i]['text'])
        temp=reviewsTEXT_clean[i]['text_split']
        index_list=[p for p,x in enumerate(temp) if x=='cocktail']
        for index in index_list:
            if (index-2 >= 0) and (temp[index-2] in pos):
                count_pos+=1
                list_pos.append(temp[index-2])
            elif (index-2 >= 0) and (temp[index-2] in neg):
                count_neg+=1
                list_neg.append(temp[index-2])           
            if (index-1 >= 0) and (temp[index-1] in pos):
                count_pos+=1
                list_pos.append(temp[index-1])
            elif (index-1 >= 0) and (temp[index-1] in neg):
                count_neg+=1
                list_neg.append(temp[index-1])            
            if (index+2 <= len(temp)) and (temp[index+1] in pos):
                count_pos+=1
                list_pos.append(temp[index+1])
            elif (index+2 <= len(temp)) and (temp[index+1] in neg):
                count_neg+=1
                list_neg.append(temp[index+1])           
            if (index+3 <= len(temp)) and (temp[index+2] in pos):
                count_pos+=1
                list_pos.append(temp[index+2])
            elif (index+3 <= len(temp)) and (temp[index+2] in neg):
                count_neg+=1
                list_neg.append(temp[index+2])


In [13]:
print('top5 positive adj near \'cocktail\' is ',list(nltk.FreqDist(list_pos).keys())[0:5])
print('top5 negative adj near \'cocktail\' is ',list(nltk.FreqDist(list_neg).keys())[0:5])

top5 positive adj near 'cocktail' is  ['great', 'well', 'best', 'good', 'pleasant']
top5 negative adj near 'cocktail' is  ['old', 'cold', 'pricey', 'sour', 'small']


In [14]:
print('count of positive adj near \'cocktail\' is ',count_pos)
print('count of negative adj near \'cocktail\' is ',count_neg)

count of positive adj near 'cocktail' is  1718
count of negative adj near 'cocktail' is  102


## beer

In [15]:
count_pos=0
count_neg=0
list_pos=[]
list_neg=[]
    
for i in range(len(reviewsTEXT_clean)):
    if 'beer' in reviewsTEXT_clean[i]['text_split']:
#        print(reviewsTEXT_clean[i]['text'])
        temp=reviewsTEXT_clean[i]['text_split']
        index_list=[p for p,x in enumerate(temp) if x=='beer']
        for index in index_list:
            if (index-2 >= 0) and (temp[index-2] in pos):
                count_pos+=1
                list_pos.append(temp[index-2])
            elif (index-2 >= 0) and (temp[index-2] in neg):
                count_neg+=1
                list_neg.append(temp[index-2])           
            if (index-1 >= 0) and (temp[index-1] in pos):
                count_pos+=1
                list_pos.append(temp[index-1])
            elif (index-1 >= 0) and (temp[index-1] in neg):
                count_neg+=1
                list_neg.append(temp[index-1])            
            if (index+2 <= len(temp)) and (temp[index+1] in pos):
                count_pos+=1
                list_pos.append(temp[index+1])
            elif (index+2 <= len(temp)) and (temp[index+1] in neg):
                count_neg+=1
                list_neg.append(temp[index+1])           
            if (index+3 <= len(temp)) and (temp[index+2] in pos):
                count_pos+=1
                list_pos.append(temp[index+2])
            elif (index+3 <= len(temp)) and (temp[index+2] in neg):
                count_neg+=1
                list_neg.append(temp[index+2])

In [16]:
print('top5 positive adj near \'beer\' is ',list(nltk.FreqDist(list_pos).keys())[0:5])
print('top5 negative adj near \'beer\' is ',list(nltk.FreqDist(list_neg).keys())[0:5])

top5 positive adj near 'beer' is  ['giant', 'well', 'huge', 'special', 'legit']
top5 negative adj near 'beer' is  ['pricey', 'cold', 'random', 'sour', 'usual']


In [17]:
print('count of positive adj near \'beer\' is ',count_pos)
print('count of negative adj near \'beer\' is ',count_neg)

count of positive adj near 'beer' is  2882
count of negative adj near 'beer' is  431


## craft beer

In [18]:
count_pos=0
count_neg=0
list_pos=[]
list_neg=[]
    
for i in range(len(reviewsTEXT_clean)):
    if 'craft beer' in reviewsTEXT_clean[i]['text']:
#        print(reviewsTEXT_clean[i]['text'])
        temp=reviewsTEXT_clean[i]['text_split']
        index_list=[p for p,x in enumerate(temp) if x=='craft']
        for index in index_list:
            if (index-2 >= 0) and (temp[index-2] in pos):
                count_pos+=1
                list_pos.append(temp[index-2])
            elif (index-2 >= 0) and (temp[index-2] in neg):
                count_neg+=1
                list_neg.append(temp[index-2])
            
            if (index-1 >= 0) and (temp[index-1] in pos):
                count_pos+=1
                list_pos.append(temp[index-1])
            elif (index-1 >= 0) and (temp[index-1] in neg):
                count_neg+=1
                list_neg.append(temp[index-1])
            
            if (index+3 <= len(temp)) and (temp[index+2] in pos):
                count_pos+=1
                list_pos.append(temp[index+2])
            elif (index+3 <= len(temp)) and (temp[index+2] in neg):
                count_neg+=1
                list_neg.append(temp[index+2])
            
            if (index+4 <= len(temp)) and (temp[index+3] in pos):
                count_pos+=1
                list_pos.append(temp[index+3])
            elif (index+4 <= len(temp)) and (temp[index+3] in neg):
                count_neg+=1
                list_neg.append(temp[index+3])

In [19]:
print('top5 positive adj near \'craft beer\' is ',list(nltk.FreqDist(list_pos).keys())[0:5])
print('top5 negative adj near \'craft beer\' is ',list(nltk.FreqDist(list_neg).keys())[0:5])

top5 positive adj near 'craft beer' is  ['great', 'fun', 'good', 'decent', 'sure']
top5 negative adj near 'craft beer' is  ['old', 'cold', 'small', 'usual']


In [20]:
print('count of positive adj near \'craft beer\' is ',count_pos)
print('count of negative adj near \'craft beer\' is ',count_neg)

count of positive adj near 'craft beer' is  177
count of negative adj near 'craft beer' is  11


## draft beer

In [21]:
count_pos=0
count_neg=0
list_pos=[]
list_neg=[]
    
for i in range(len(reviewsTEXT_clean)):
    if 'draft beer' in reviewsTEXT_clean[i]['text']:
#        print(reviewsTEXT_clean[i]['text'])
        temp=reviewsTEXT_clean[i]['text_split']
        index_list=[p for p,x in enumerate(temp) if x=='draft']
        for index in index_list:
            if (index-2 >= 0) and (temp[index-2] in pos):
                count_pos+=1
                list_pos.append(temp[index-2])
            elif (index-2 >= 0) and (temp[index-2] in neg):
                count_neg+=1
                list_neg.append(temp[index-2])
            
            if (index-1 >= 0) and (temp[index-1] in pos):
                count_pos+=1
                list_pos.append(temp[index-1])
            elif (index-1 >= 0) and (temp[index-1] in neg):
                count_neg+=1
                list_neg.append(temp[index-1])
            
            if (index+3 <= len(temp)) and (temp[index+2] in pos):
                count_pos+=1
                list_pos.append(temp[index+2])
            elif (index+3 <= len(temp)) and (temp[index+2] in neg):
                count_neg+=1
                list_neg.append(temp[index+2])
            
            if (index+4 <= len(temp)) and (temp[index+3] in pos):
                count_pos+=1
                list_pos.append(temp[index+3])
            elif (index+4 <= len(temp)) and (temp[index+3] in neg):
                count_neg+=1
                list_neg.append(temp[index+3])

In [22]:
print('top5 positive adj near \'draft beer\' is ',list(nltk.FreqDist(list_pos).keys())[0:5])
print('top5 negative adj near \'draft beer\' is ',list(nltk.FreqDist(list_neg).keys())[0:5])

top5 positive adj near 'draft beer' is  ['great', 'nice', 'best', 'famous', 'good']
top5 negative adj near 'draft beer' is  ['cold', 'small', 'bad']


In [23]:
print('count of positive adj near \'draft beer\' is ',count_pos)
print('count of negative adj near \'draft beer\' is ',count_neg)

count of positive adj near 'draft beer' is  58
count of negative adj near 'draft beer' is  14


## beer select

In [24]:
count_pos=0
count_neg=0
list_pos=[]
list_neg=[]
    
for i in range(len(reviewsTEXT_clean)):
    if 'beer select' in reviewsTEXT_clean[i]['text']:
#        print(reviewsTEXT_clean[i]['text'])
        temp=reviewsTEXT_clean[i]['text_split']
        index_list=[p for p,x in enumerate(temp) if x=='select']
        for index in index_list:
            if (index-3 >= 0) and (temp[index-3] in pos):
                count_pos+=1
                list_pos.append(temp[index-3])
            elif (index-3 >= 0) and (temp[index-3] in neg):
                count_neg+=1
                list_neg.append(temp[index-3])
            
            if (index-2 >= 0) and (temp[index-2] in pos):
                count_pos+=1
                list_pos.append(temp[index-2])
            elif (index-2 >= 0) and (temp[index-2] in neg):
                count_neg+=1
                list_neg.append(temp[index-2])
            
            if (index+2 <= len(temp)) and (temp[index+1] in pos):
                count_pos+=1
                list_pos.append(temp[index+1])
            elif (index+2 <= len(temp)) and (temp[index+1] in neg):
                count_neg+=1
                list_neg.append(temp[index+1])
            
            if (index+3 <= len(temp)) and (temp[index+2] in pos):
                count_pos+=1
                list_pos.append(temp[index+2])
            elif (index+3 <= len(temp)) and (temp[index+2] in neg):
                count_neg+=1
                list_neg.append(temp[index+2])
            

In [25]:
print('top5 positive adj near \'beer select\' is ',list(nltk.FreqDist(list_pos).keys())[0:5])
print('top5 negative adj near \'beer select\' is ',list(nltk.FreqDist(list_neg).keys())[0:5])

top5 positive adj near 'beer select' is  ['great', 'good', 'well', 'decent', 'huge']
top5 negative adj near 'beer select' is  ['small', 'weak', 'cant', 'ultra', 'monstrous']


In [26]:
print('count of positive adj near \'beer select\' is ',count_pos)
print('count of negative adj near \'beer select\' is ',count_neg)

count of positive adj near 'beer select' is  697
count of negative adj near 'beer select' is  13
